# Pytorch

## Instalaciones

In [ ]:
!pip install torch
!pip install torchvision

## Importar dependencias

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Definir Hiperparámetros

In [ ]:
input_size = 784
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 100
lr = 1e-3

## Descargando la base de datos mnist

In [ ]:
train_data = dsets.FashionMNIST(root='./data', train=True,
                                transform=transforms.ToTensor(), download=True)

test_data = dsets.FashionMNIST(root='./data', train=False,
                               transform=transforms.ToTensor())

## Leyendo la data

In [ ]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

## Definir modelo

In [ ]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.drop(out)
    out = self.fc2(out)
    return out

## Instancia del modelo

In [ ]:
net = Net(input_size, hidden_size, num_classes)

if torch.cuda.is_available():
  net.cuda()

## Compilación

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

## Entrenamiento

In [ ]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print('Epoca [%d/%d], Step [%d/%d], Loss: %.4f'
              % (epoch + 1, num_epochs, i + 1, len(train_data) // batch_size, loss.item()))


Epoca [1/5], Step [100/600], Loss: 0.7294
Epoca [1/5], Step [200/600], Loss: 0.6229
Epoca [1/5], Step [300/600], Loss: 0.5962
Epoca [1/5], Step [400/600], Loss: 0.6795
Epoca [1/5], Step [500/600], Loss: 0.4066
Epoca [1/5], Step [600/600], Loss: 0.3599
Epoca [2/5], Step [100/600], Loss: 0.4830
Epoca [2/5], Step [200/600], Loss: 0.4294
Epoca [2/5], Step [300/600], Loss: 0.3679
Epoca [2/5], Step [400/600], Loss: 0.3294
Epoca [2/5], Step [500/600], Loss: 0.3733
Epoca [2/5], Step [600/600], Loss: 0.4362
Epoca [3/5], Step [100/600], Loss: 0.4667
Epoca [3/5], Step [200/600], Loss: 0.3739
Epoca [3/5], Step [300/600], Loss: 0.3629
Epoca [3/5], Step [400/600], Loss: 0.4732
Epoca [3/5], Step [500/600], Loss: 0.4111
Epoca [3/5], Step [600/600], Loss: 0.1678
Epoca [4/5], Step [100/600], Loss: 0.2814
Epoca [4/5], Step [200/600], Loss: 0.4091
Epoca [4/5], Step [300/600], Loss: 0.3110
Epoca [4/5], Step [400/600], Loss: 0.2562
Epoca [4/5], Step [500/600], Loss: 0.3436
Epoca [4/5], Step [600/600], Loss:

## Evaluación del modelo en los datos de prueba

In [ ]:
correct = 0
total = 0
for images, labels in test_gen:
    images = images.view(-1, 28 * 28).cuda()  # Aplanar las imágenes
    labels = labels.cuda()

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()  # Convertir a escalar
    total += labels.size(0)

# Calcular la precisión (accuracy)
print('Accuracy: %.3f %%' % (100 * correct / total))

Accuracy: 86.160 %
